In [1]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
!pip install transformers[torch] datasets torch peft trl==0.4.7 bitsandbytes accelerate cuda-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00


In [ ]:
!zip -r ./lora_model_2.zip ./lora_model
# pass

In [ ]:
# !unzip ./lora_model.zip
# pass

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
# Init imports and constants

import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, AutoTokenizer, pipeline
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, get_peft_model

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("dair-ai/emotion", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# Loading tokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [5]:
# Loading model and formatting dataset

quant_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)

def merge_columns(example):
  emotion_mappings = {
      0: "sadness",
      1: "joy",
      2: "love",
      3: "anger",
      4: "fear",
      5: "surprise"
  }

  merged = example["text"] + " ->: " + emotion_mappings[example["label"]]

  messages = [
      {
          "role": "user",
          "content": example["text"] + " ->: "
      },
      {
          "role": "assistant",
          "content": merged
      }
  ]

  example["prediction"] = messages

  return example

predictions = dataset.map(merge_columns)
formatted_dataset = predictions.map(lambda row: {"formatted": tokenizer.apply_chat_template(row["prediction"], tokenize=False, add_generation_prompt=False)})

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [ ]:
print(base_model.get_memory_footprint() / 1**9)

In [6]:
# LoRa configuration

lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

lora_model = get_peft_model(base_model, lora_config)

In [7]:
# Train LoRa model

training_arguments = TrainingArguments(
    output_dir="training_output",
    num_train_epochs=3,
    logging_steps=100,
    save_steps=100,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,
    warmup_steps=100,
)

trainer = SFTTrainer(
    model=lora_model,
    train_dataset=formatted_dataset,
    dataset_text_field="formatted",
    args=training_arguments
)

trainer.train()
trainer.save_model("./lora_model")

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,2.168800
200,1.316700
300,1.139500
400,1.116300
500,1.115500
600,1.110400
700,1.100800


In [8]:
# Merge LoRa and base model to create fine tuned model

full_base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    trust_remote_code=True,
)

lora_adapter = PeftModel.from_pretrained(
    full_base_model,
    "./lora_model"
)

fine_tuned_model = lora_adapter.merge_and_unload()

fine_tuned_model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/tokenizer.json')

In [10]:
# Testing original model

text_gen = pipeline(
    "text-generation",
    model=BASE_MODEL,
)

messages = [
    {
        "role": "user",
        "content": "i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: ",
    },
]

prompt = text_gen.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
result = text_gen(prompt)
print(result[0]["generated_text"])

<|user|>
i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: </s>
<|assistant|>
You may still feel nostalgic about the fireplace that is still on the property, even though it has been out of use for some time. The fireplace was a cherished part of your childhood and memories of playing with your siblings and family around it may still linger. While it may be a bittersweet reminder of the past, it is still a part of the property and can be enjoyed as a decorative feature or as a functional heating source.


In [11]:
# Testing fine tuned model

fine_text_gen = pipeline(
    "text-generation",
    model=fine_tuned_model,
    tokenizer=tokenizer
)

messages = [
    {
        "role": "user",
        "content": "i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: ",
    },
]

prompt = fine_text_gen.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
result = fine_text_gen(prompt)
print(result[0]["generated_text"])

<|user|>
i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: </s>
<|assistant|>
i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: sadness
i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: sadness
i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: sadness
i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: sadness
i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: sadness
i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: sadness
i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: sadness
i am ever feeling nostalgic about the fireplace i will know that it is still on the property ->: sadness
i am ever feeling nostalgic about t